In [ ]:
#export
from local.imports import *
from local.test import *
from local.core import *
from local.data.all import *
from local.notebook.showdoc import show_doc

In [ ]:
adult_source = untar_data(URLs.ADULT_SAMPLE)
df = pd.read_csv(adult_source/'adult.csv')

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']

In [ ]:
df.head()

## Preprocessing

In [ ]:
means,stds = {},{}
for n in cont_names:
    if pd.isnull(df[n]).any(): 
        df[n+"_na"] = pd.isnull(df[n])
        cat_names.append(n+'_na')
        df[n] = df[n].fillna(df[n].median())
    means[n],stds[n] = (df[n].mean(),df[n].std())
    df[n] = (df[n] - means[n]) / stds[n]

In [ ]:
cat_names

In [ ]:
for n in cat_names + ["salary"]: df[n] = pd.Categorical(df.loc[:,n], ordered=True)

In [ ]:
cats  = df[cat_names]
conts = df[cont_names]
y = df["salary"]

In [ ]:
cats.head()

In [ ]:
conts.head()

## DataBunch

In [ ]:
class DFDataset(GetAttr):
    _xtra = ["__len__"]
    def __init__(self, cats, conts, y): 
        self.y,self.conts,self.cats = y,conts,cats
        self.default = y
    def __getitem__(self, i): 
        return ((self.cats.iloc[i].values,self.conts.iloc[i].values), self.y[i])

In [ ]:
cats = df.loc[:, cat_names]
for n in cat_names: cats[n] = cats[n].cat.codes
conts = df[cont_names]

In [ ]:
ds = DFDataset(cats, conts, y.cat.codes)

In [ ]:
ds[0]

In [ ]:
dl = TfmdDL(ds)

In [ ]:
dl.one_batch()

## With rapids

In [ ]:
import cudf
import nvcategory

In [ ]:
df = pd.read_csv(adult_source/'adult.csv')
df = cudf.from_pandas(df)

In [ ]:
cat_names = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race']
cont_names = ['age', 'fnlwgt', 'education-num']

In [ ]:
means,stds = {},{}
for n in cont_names:
    if df[n].isnull().any(): 
        df[n+"_na"] = df[n].isnull()
        cat_names.append(n+'_na')
        df[n] = df[n].fillna(df[n].mean()) #TODO: request median
    means[n],stds[n] = (df[n].mean(),df[n].std())
    df[n] = (df[n] - means[n]) / stds[n]

In [ ]:
cat_names

In [ ]:
classes = {}
for n in cat_names + ["salary"]: 
    if df[n].dtype != "object": df[n] = df[n].astype("str")
    classes[n] = nvcategory.from_strings(df[n].data).keys()
    df[n] = nvcategory.from_strings(df[n].data).set_keys(classes[n]).values()

In [ ]:
cats  = df[cat_names]
conts = df[cont_names]
y = df["salary"]

In [ ]:
print(cats.head())

In [ ]:
print(conts.head())

## DataBunch TODO

In [ ]:
class DFDataset(GetAttr):
    _xtra = ["__len__"]
    def __init__(self, cats, conts, y): 
        self.y,self.conts,self.cats = y,conts,cats
        self.default = y
    def __getitem__(self, i): 
        return ((self.cats.iloc[i].values,self.conts.iloc[i].values), self.y[i])

In [ ]:
cats = df.loc[:, cat_names]
for n in cat_names: cats[n] = cats[n].cat.codes
conts = df[cont_names]

In [ ]:
ds = DFDataset(cats, conts, y.cat.codes)

In [ ]:
ds[0]

In [ ]:
dl = TfmdDL(ds)

In [ ]:
dl.one_batch()